Importing libraries and defining functions

In [1]:
import numpy as np
import os
import rasterio
from scipy.ndimage import label
from PIL import Image
from scipy.stats import mode
from rasterio.transform import from_origin

Image.MAX_IMAGE_PIXELS = 933120000


def classAssignment(objectArray, classificationCrop, min_size=3, connectivity=1):
    # Iterate over each unique object label (ignoring 0)
    unique_objects = np.unique(objectArray)
    unique_objects = unique_objects[unique_objects != 0]

    for obj in unique_objects:
        # Create a binary mask for the current object
        objectMask = (objectArray == obj)
        
        # Label the connected components within this mask
        labeled_array, num_features = label(objectMask, structure=np.ones((3, 3)) if connectivity == 1 else np.ones((3, 3)))

        for comp_label in range(1, num_features + 1):
            # Create a mask for the current connected component
            comp_mask = (labeled_array == comp_label)
            comp_size = np.sum(comp_mask)
            
            # Ignore components smaller than min_size
            if comp_size < min_size:
                continue
            
            # Find the mode of the corresponding classificationCrop values
            f = np.multiply(comp_mask, classificationCrop).ravel()
            h = mode(f[f != 0])[0]
            if h.shape == (0,):
                continue
            # Replace the object in classificationCrop with the mode value
            classificationCrop = np.where(comp_mask, h, classificationCrop)

    return classificationCrop





def create_geotiff_from_array(numpy_array, output_geotiff_path):
    # Fixed georeferencing information - change according to your situation
    transform = from_origin(west=491880.0, north=5590350.0, xsize=10, ysize=10)
    crs = {'init': 'epsg:32633'}  # UTM 33n - Change according your EPSG

    # get array dimensions
    height, width = numpy_array.shape

    # GeoTIFF file save
    with rasterio.open(output_geotiff_path, 'w', driver='GTiff', height=height, width=width, count=1, dtype=numpy_array.dtype, crs=crs, transform=transform, compress='lzw') as dst:
        dst.write(numpy_array, 1)





 Update layer from full-stride tiles:

In [ ]:
def objectMatrix(objects, areas, scoreInput,IOUInput):
    objectsArray = np.zeros_like(objects[:,:,0])
    objectsArea = np.ones_like(objects[:,:,0])*10000
    objectsScore = np.zeros_like(objects[:,:,0])
    objectsIOU = np.zeros_like(objects[:,:,0])

    for i in range(objects.shape[2]):
        selectedObject = objects[:,:,i]
        objectArea = areas[0,0,i]
        if (np.sum(selectedObject) != objectArea) or (objectArea>10000): #If the object is larger than 10000 pixels, its processing is skipped
            continue
        
        score = scoreInput[0,0,i]
        if score < scoreTreshold:
            continue
        
        IOU = IOUInput[0,0,i]

        objectArea = selectedObject * objectArea

        objectsArea = np.minimum(np.where(objectArea == 0, 10001, objectArea),objectsArea)
        objectsArray = np.where(objectArea == objectArea, i+1, objectsArray)
        objectsScore = np.where(objectArea == objectArea, score, objectsScore)
        objectsIOU = np.where(objectArea == objectArea, IOU, objectsIOU)
        
    return objectArray, objectScore, objectIOU

     
scoreTreshold = 50

classification = (np.array(Image.open('classification.tif'),dtype=np.uint8))[:,:]

objectsLayer = np.zeros_like(classification, dtype=np.uint16)
objectScore= np.zeros_like(classification)
objectIOU= np.zeros_like(classification)

# Get a list of files containing objects in a folder
fileList = sorted(os.listdir("objects/"))

# tile size of objects
size = 400

# object files iteration 
for file in fileList:
    loaded_data = np.load("objects/" + file)

    # splitting the file name to get the relative position of objects
    nameSplit = file[:-4].split("_")
    x = (int(nameSplit[0]))
    y = (int(nameSplit[1]))

    # Process only tiles that do not overlap
    if x % size != 0:
        continue
    if y % size != 0:
        continue

    classificationCrop = classification[x:x+size,y:y+size]

    matrix = loaded_data['objects']

    area = loaded_data['area']
    score = loaded_data['SS']
    IOU = loaded_data['IOU']


    newObjects = objectMatrix(matrix,area,score,IOU)

    objectsLayer[x:x+size,y:y+size] = newObjects[0]
    objectScore[x:x+size,y:y+size] = newObjects[1]
    objectIOU[x:x+size,y:y+size] = newObjects[2]


    updatedClassification = classAssignment(newObjects[0], classificationCrop)

    classification[x:x+size,y:y+size] = updatedClassification



# storing arrays in geotifs
create_geotiff_from_array(objectsLayer, 'Objects.tif')
create_geotiff_from_array(classification, 'Classification.tif')
create_geotiff_from_array(objectScore, 'Score.tif')




Update layer from half-stride tiles:

In [ ]:
def objectMatrix(objects, areas, scoreInput,IOUInput,coords,objectArea):
    objectsArray = np.zeros_like(objects[:,:,0])
    objectsScore = np.zeros_like(objects[:,:,0])
    objectsIOU = np.zeros_like(objects[:,:,0])

    for i in range(objects.shape[2]):
        
        score = scoreInput[0,0,i]
        if score < scoreTreshold:
            continue

        selectedObject = objects[:,:,i]
        objectArea = areas[0,0,i]
        if (np.sum(selectedObject) != objectArea) or (objectArea>10000): #If the object is larger than 10000 pixels, its processing is skipped
            continue     
        
        IOU = IOUInput[0,0,i]

        objectArea = selectedObject * objectArea
    
        objectsArea = np.minimum(np.where(objectArea == 0, 10001, objectArea),objectArea)
        objectsArray = np.where(objectArea == objectArea, i+1, objectArray)
        objectsScore = np.where(objectArea == objectArea, score, objectScore)
        objectsOU = np.where(objectArea == objectArea, IOU, objectIOU)
    return objectArray, objectScore, objectIOU, objectArea






classificationBase = (np.array(Image.open('classification.tif'),dtype=np.uint8))[:,:]

# tile size of objects
size = 400

scoreTreshold = 50

classification = np.copy(classificationBase)

objectsLayer = np.zeros_like(classification, dtype=np.uint16)
objectScore= np.zeros_like(classification)
objectIOU= np.zeros_like(classification)
areaArray = np.ones_like(classification, dtype=np.uint16)*10001 #10,001 = null (NoData)

# Get a list of files containing objects in a folder
fileList = sorted(os.listdir('objects/'))

# object files iteration
for file in fileList:
    
    loaded_data = np.load('objects/' + file)
    # splitting the file name to get the relative position of objects
    nameSplit = file[:-4].split("_")
    x = (int(nameSplit[0]))
    y = (int(nameSplit[1]))

    matrix = loaded_data['objects']
    area = loaded_data['area']
    score = loaded_data['SS']
    IOU = loaded_data['IOU']
    coords = loaded_data['coords']

    newObjects = objectMatrix(matrix,area,score,IOU,coords,areaArray[x:x+size,y:y+size])

    objectsLayer[int(x+size/4):int(x+size/4*3),int(y+size/4):int(y+size/4*3)] = newObjects[0][int(size/4):int(size/4*3),int(size/4):int(size/4*3)]
    objectScore[int(x+size/4):int(x+size/4*3),int(y+size/4):int(y+size/4*3)] = newObjects[1][int(size/4):int(size/4*3),int(size/4):int(size/4*3)]
    objectIOU[int(x+size/4):int(x+size/4*3),int(y+size/4):int(y+size/4*3)] = newObjects[2][int(size/4):int(size/4*3),int(size/4):int(size/4*3)]
    areaArray[int(x+size/4):int(x+size/4*3),int(y+size/4):int(y+size/4*3)] = newObjects[3][int(size/4):int(size/4*3),int(size/4):int(size/4*3)]

    updatedClassification = classAssignment(newObjects[0], np.copy(classificationBase[x:x+size,y:y+size]))
    classification[int(x+size/4):int(x+size/4*3),int(y+size/4):int(y+size/4*3)] = updatedClassification[int(size/4):int(size/4*3),int(size/4):int(size/4*3)]

# storing arrays in geotifs
create_geotiff_from_array(objectsLayer, 'Objects.tif')
create_geotiff_from_array(classification, 'Clasification.tif')
create_geotiff_from_array(objectScore, 'Score.tif')
create_geotiff_from_array(areaArray, 'Area.tif')
